In [1]:
import os

# Option 1 (Recommended): Test environment variable reading
api_key_test_env = os.getenv("OPENROUTER_API_KEY")
print(f"API Key (from env var): '{api_key_test_env}'")

# Option 2 (For temporary direct assignment if env var fails):
api_key_test_direct = "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" # PASTE YOUR ACTUAL KEY HERE
print(f"API Key (direct assignment): '{api_key_test_direct}'")

# Then, verify length (should be around 64-68 characters for OpenRouter v1 keys)
if api_key_test_env: # Or api_key_test_direct if you uncomment that line
    print(f"API Key length: {len(api_key_test_env)}")
else:
    print("API Key is None or empty.")

API Key (from env var): 'None'
API Key (direct assignment): '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'
API Key is None or empty.


In [2]:
# Stap 0: Initialisatie en Configuratie (Aangepast voor OpenRouter)

import pdfplumber
import re
import requests
import os
import time
from PIL import Image
import pytesseract
from pdf2image import convert_from_path

# Configureer je OpenRouter API sleutel.
# Optie 2: Hardcode de sleutel hier (MINDER VEILIG, ALLEEN VOOR TESTEN!)
OPENROUTER_API_KEY = "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" # <-- JE NIEUWE, GELDIGE KEY # <-- DIRECT DE KEY HIER INVULLEN

# Verwijder of commentarieer de volgende regels als je hardcodet:
# if not OPENROUTER_API_KEY:
#     raise ValueError("OpenRouter API Key is niet ingesteld. Stel 'OPENROUTER_API_KEY' in als omgevingsvariabele.")

# OpenRouter API URL (dit is de algemene URL voor OpenRouter's chat completions)
OPENROUTER_API_URL = "https://openrouter.ai/api/v1/chat/completions"

# Het SPECIFIEKE DeepSeek model dat je via OpenRouter wilt gebruiken
MODEL_NAME = "deepseek/deepseek-r1:free"

print("Configuratie geladen voor OpenRouter.")

Configuratie geladen voor OpenRouter.


In [3]:
# Stap 1: Definieer de lijst van te verwerken PDF-bestanden

# Maak hier een lijst van alle PDF-paden die je wilt analyseren.
# Voeg zoveel paden toe als je nodig hebt.
pdf_file_paths = [
    "/Users/mohamedelharchaoui/Downloads/Assignment 3/0083_01-01-2025 tot 01-07-2026_akkoord.pdf",
    "/Users/mohamedelharchaoui/Downloads/Assignment 3/0182_01-01-2025 tot 01-01-2027_akkoord.pdf",
    "/Users/mohamedelharchaoui/Downloads/Assignment 3/1069_01-01-2025 tot 01-01-2027_akkoord.pdf",
    "/Users/mohamedelharchaoui/Downloads/Assignment 3/2070_01-01-2025 tot 01-01-2028_akkoord.pdf"
    # Voeg hier meer paden toe, bijvoorbeeld:
    # "/pad/naar/jouw/andere_cao.pdf"
]

print(f"{len(pdf_file_paths)} PDF-bestanden gedefinieerd voor verwerking.")

# --- Functiedefinities voor text-extractie ---

def extract_text_from_pdf_with_ocr(pdf_path, min_text_length=100):
    """
    Probeert eerst tekst digitaal te extraheren. Als dat te weinig tekst oplevert,
    schakelt het automatisch over naar OCR met Tesseract.
    """
    text = ""
    # Eerste poging: digitale extractie met pdfplumber
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
        print("Methode: Digitale extractie (pdfplumber) geslaagd.")
    except Exception as e:
        print(f"Fout tijdens digitale extractie: {e}. Probeert nu OCR.")
        text = "" # Reset de tekst voor de OCR poging

    # Controleer of de digitale extractie genoeg tekst heeft opgeleverd
    if len(text.strip()) >= min_text_length:
        return text

    # Tweede poging: OCR met Tesseract voor ingescande PDF's
    print(f"Digitale extractie leverde < {min_text_length} tekens op. Overschakelen naar OCR...")
    try:
        # Zet de PDF-pagina's om naar afbeeldingen
        images = convert_from_path(pdf_path)
        ocr_text = ""
        for i, image in enumerate(images):
            print(f"  Verwerken van pagina {i+1} met OCR...")
            # Gebruik Tesseract om tekst van de afbeelding te lezen (specificeer de Nederlandse taal)
            ocr_text += pytesseract.image_to_string(image, lang='nld') + "\n"
        print("Methode: OCR-extractie (Tesseract) geslaagd.")
        return ocr_text
    except Exception as e:
        print(f"Fout tijdens OCR-extractie: {e}")
        return None

4 PDF-bestanden gedefinieerd voor verwerking.


In [4]:
# Stap 2: Gebruik van regex om zinnen met percentages te extraheren

def extract_percentage_sentences(text):
    """
    Zoekt naar zinnen in de tekst die percentages bevatten,
    met een focus op contexten die gerelateerd kunnen zijn aan loonstijgingen.
    """
    if not text:
        return []

    # Gecorrigeerd regex patroon. '[^.?!]' wordt gebruikt om de hele zin te vangen.
    percentage_pattern = re.compile(
        r'([^.?!]*?(?:loon|salaris|cao|verhoging|stijging|toeslag)\s*[^.?!]*?\d[\d.,]*\s?%\s*[^.?!]*?[.?!])',
        re.IGNORECASE
    )
    sentences_with_percentage = percentage_pattern.findall(text)

    # Verwijder overtollige spaties en vervang de ECHTE newline karakters ('\n')
    clean_sentences = [s.strip().replace('\n', ' ') for s in sentences_with_percentage]
    return clean_sentences

In [5]:
import json
from json import JSONDecoder

def classify_with_deepseek(sentence, api_key, api_url, model_name, max_retries=3, delay=5):
    """
    Roept de API aan via OpenRouter om een zin te classificeren en om uitleg te vragen.
    Is nu in staat om MEERDERE loonstijgingen uit één zin te extraheren en is robuust tegen JSON-fouten.
    """
    if not api_key:
        print("Fout: OpenRouter API-sleutel is leeg of niet ingesteld.")
        return None

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
        "HTTP-Referer": "https://localhost/sakkal-cao-analyzer",
        "X-Title": "Team Sakkal CAO Analyzer"
    }
    
    # Finale prompt, nu met een extra voorbeeld om referentiedatums te negeren.
    system_prompt = """Je bent een expert in het analyseren van Nederlandse CAO-teksten. Je taak is om te bepalen of een zin concrete, definitieve loonstijgingen beschrijft en **alle** details daarvan te extraheren.

**Instructies:**
- Een zin kan **meerdere** loonstijgingen bevatten (bijv. in een opsomming). Je moet ze **allemaal** vinden.
- Classificeer alleen **daadwerkelijke, gegarandeerde afspraken** over salarisverhogingen als 'Loonstijging'.
- **Negeer** zinnen die beginnen met 'Rekenvoorbeeld', 'Voorbeeld:', 'Stel dat', of 'Berekening'.
- **Negeer** structurele salarisverschillen tussen functies (bv. 'Level 2 is 10% hoger dan Level 1').
- **Cruciaal: Negeer voorwaardelijke verhogingen** die afhankelijk zijn van een conditie (bv. 'tenzij', 'indien').
- **Cruciaal: Negeer vergelijkende berekeningen.** Als een stijging wordt beschreven 'ten opzichte van' een datum in het verleden, is dit een vergelijking, geen nieuwe afspraak.

**Jouw taak:**
Analyseer de zin en geef een JSON-object terug met DRIE sleutels:
1. 'classificatie': 'Loonstijging' of 'Geen Loonstijging'.
2. 'verhogingen': Een **lijst** van JSON-objecten. Elk object bevat 'datum' en 'percentage'. Als er geen gegarandeerde loonstijgingen zijn, moet deze lijst leeg zijn: `[]`.
3. 'uitleg': een korte, duidelijke toelichting op je keuze.

**BELANGRIJK:** Je antwoord MOET **uitsluitend** een enkel, geldig JSON-object zijn. Geen extra tekst."""

    # Gebruik van triple quotes (''') voor de JSON-strings om escaping-fouten te voorkomen.
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "De salarissen stijgen met 2% op 01-01-2025 en met nog eens 3% op 01-07-2025."},
        {"role": "assistant", "content": '''{"classificatie": "Loonstijging", "verhogingen": [{"datum": "01/01/2025", "percentage": 2.0}, {"datum": "01/07/2025", "percentage": 3.0}], "uitleg": "De zin bevat twee concrete loonstijgingen."}'''},
        {"role": "user", "content": "Met ingang van 1 januari 2025 worden deze verlofdagen omgezet in een structurele salarisverhoging van 0,84% tenzij werkgever deze verlofdagen al eerder heeft omgezet."},
        {"role": "assistant", "content": '''{"classificatie": "Geen Loonstijging", "verhogingen": [], "uitleg": "De verhoging is voorwaardelijk vanwege de 'tenzij'-clausule en dus niet gegarandeerd."}'''},
        # Nieuw, cruciaal voorbeeld voor het negeren van referentiedatums.
        {"role": "user", "content": "het eindloon stijgt ten opzichte van het eindloon op 31 december 2024 met 6,9%."},
        {"role": "assistant", "content": '''{"classificatie": "Geen Loonstijging", "verhogingen": [], "uitleg": "Dit is een vergelijkende berekening ten opzichte van een referentiedatum, geen nieuwe loonafspraak met een ingangsdatum."}'''},
        {"role": "user", "content": sentence}
    ]

    data = { "model": model_name, "messages": messages, "response_format": { "type": "json_object" }, "max_tokens": 400, "temperature": 0.0 }
    
    for attempt in range(max_retries):
        try:
            response = requests.post(api_url, json=data, headers=headers, timeout=45)
            response.raise_for_status()

            # Robuuste JSON-parsing die extra data na een geldig object negeert
            try:
                content = response.text
                full_response_json = json.loads(content)
                message_content = full_response_json['choices'][0]['message']['content']
                
                # --- FINALE ROBUUSTE PARSING (VERBETERD) ---
                # Zoek het EERSTE '{' en het LAATSTE '}' om het volledige JSON-object te vangen.
                # Dit is robuuster, omdat de AI soms extra tekst na het JSON-object toevoegt.
                start_index = message_content.find('{')
                end_index = message_content.rfind('}')
                
                if start_index != -1 and end_index != -1 and end_index > start_index:
                    json_str = message_content[start_index : end_index + 1]
                    try:
                        # Probeer het geïsoleerde JSON-blok te parsen
                        return json.loads(json_str)
                    except json.JSONDecodeError as e:
                        print(f"Fout bij parsen van geïsoleerd JSON-object: {e}")
                        print(f"Onbewerkte content van AI: {message_content}")
                        return None # Keer terug als het parsen mislukt
                else:
                    # Dit gebeurt als er geen '{' of '}' is gevonden.
                    print("Geen JSON-object gevonden in de AI-respons.")
                    print(f"Onbewerkte content van AI: {message_content}")
                    return None
                # --- EINDE FINALE PARSING ---

            except (json.JSONDecodeError, KeyError, IndexError, TypeError) as e:
                print(f"Fout bij parsen van de volledige API-respons: {e}")
                print(f"Ontvangen tekst: {response.text}")
                return None
        except requests.exceptions.RequestException as e:
            print(f"Netwerkfout: {e}")
            if attempt < max_retries - 1:
                time.sleep(delay)
            else:
                return None
            
    print(f"Fout na {max_retries} pogingen.")
    return None

In [6]:
# Stap 4: Hoofd-loop voor het verwerken van alle gedefinieerde PDF's

from datetime import datetime
import json

# Dictionary om de eindresultaten van alle PDF's op te slaan
final_json_output = {}

# Loop door elk opgegeven PDF-pad
for pdf_path in pdf_file_paths:
    pdf_filename = os.path.basename(pdf_path)
    print("\n" + "="*80)
    print(f"--- Begin analyse van: {pdf_filename} ---\n")
    
    # Stap 1: Tekst extractie
    extracted_text = extract_text_from_pdf_with_ocr(pdf_path)
    if not extracted_text:
        print(f"Kon geen tekst extraheren uit {pdf_filename}. Bestand wordt overgeslagen.")
        final_json_output[pdf_filename] = {"error": "Tekstextractie mislukt", "verhogingen": []}
        continue

    # Stap 2: Zinnen met percentages vinden
    sentences = extract_percentage_sentences(extracted_text)
    if not sentences:
        print(f"Geen relevante zinnen met percentages gevonden in {pdf_filename}.")
        final_json_output[pdf_filename] = {"error": "Geen relevante zinnen gevonden", "verhogingen": []}
        continue

    print(f"{len(sentences)} zinnen gevonden voor analyse.")
    
    # Stap 3: Classificatie en verzameling
    all_found_increases = []
    print("\nBeginnen met classificatie via DeepSeek API:")
    for i, sentence in enumerate(sentences):
        short_sentence = (sentence[:120] + '...') if len(sentence) > 120 else sentence
        print(f"\nVerwerken zin {i+1}/{len(sentences)}: {short_sentence}")
        
        result_json = classify_with_deepseek(sentence, OPENROUTER_API_KEY, OPENROUTER_API_URL, MODEL_NAME)
        
        if result_json:
            classificatie = result_json.get('classificatie', 'Onbekend')
            verhogingen = result_json.get('verhogingen', [])
            uitleg = result_json.get('uitleg', '')
            
            if classificatie == 'Loonstijging' and verhogingen:
                print(f"  Resultaat: ✅ {len(verhogingen)} loonstijging(en) gevonden.")
                all_found_increases.extend(verhogingen)
            else:
                print(f"  Resultaat: ❌ Geen Loonstijging")
        else:
            print("  Resultaat: ❓ Fout bij het classificeren van de zin.")
        
        time.sleep(1)

    # Stap 4: Resultaten voor de huidige PDF groeperen en aggregeren
    grouped_increases = {}
    for increase in all_found_increases:
        datum = increase.get('datum')
        percentage = increase.get('percentage')
        if datum and percentage and datum != 'N.v.t.' and percentage != 'N.v.t.':
            if datum not in grouped_increases:
                grouped_increases[datum] = []
            if isinstance(percentage, (int, float)):
                grouped_increases[datum].append(percentage)

    aggregated_increases = []
    for datum, percentages in grouped_increases.items():
        # Sorteer percentages om de output voorspelbaar te maken
        percentages.sort()
        # Gebruik de samengevoegde string-notatie die we eerder hadden
        percentage_str = "/".join([f"{p:.2f}".replace('.', ',') + '%' for p in percentages])
        aggregated_increases.append({'datum': datum, 'percentage': percentage_str})
    
    # Sorteer de uiteindelijke lijst op datum
    def sort_key(item):
        try:
            return datetime.strptime(item.get('datum', ''), '%d/%m/%Y')
        except (ValueError, TypeError):
            return datetime.max
    aggregated_increases.sort(key=sort_key)
    
    # Voeg het resultaat toe aan de hoofd-dictionary
    final_json_output[pdf_filename] = {"verhogingen": aggregated_increases}
    print(f"\nAnalyse voor {pdf_filename} voltooid.")

# --- Aan het einde, na de loop, print de volledige JSON-output ---
print("\n" + "="*80)
print("--- Alle PDF's zijn verwerkt. Hier is de volledige JSON-output: ---")
print("="*80)
print(json.dumps(final_json_output, indent=2, ensure_ascii=False))


--- Begin analyse van: 0083_01-01-2025 tot 01-07-2026_akkoord.pdf ---

Methode: Digitale extractie (pdfplumber) geslaagd.
2 zinnen gevonden voor analyse.

Beginnen met classificatie via DeepSeek API:

Verwerken zin 1/2: Salaris Tijdens de looptijd van de cao worden de schaalsalarissen en de feitelijke salarissen bij een voltijd dienstverb...
  Resultaat: ✅ 3 loonstijging(en) gevonden.

Verwerken zin 2/2: Met ingang van 1 januari 2025 worden deze verlofdagen omgezet in een structurele salarisverhoging van 0,84% tenzij werkg...
  Resultaat: ❌ Geen Loonstijging

Analyse voor 0083_01-01-2025 tot 01-07-2026_akkoord.pdf voltooid.

--- Begin analyse van: 0182_01-01-2025 tot 01-01-2027_akkoord.pdf ---

Methode: Digitale extractie (pdfplumber) geslaagd.
8 zinnen gevonden voor analyse.

Beginnen met classificatie via DeepSeek API:

Verwerken zin 1/8: Concreet betekent dit dat de feitelijke salarissen per 1 januari 2025 met 2,75% stijgen.
Fout bij parsen van geïsoleerd JSON-object: Extra data: l